# Extracting model results

In [1]:
import pandas as pd
from model.sirds_model_visualization import get_sirds, get_sirds_extras

## Loading data

In [2]:
df = pd.read_csv('../data/df_ts_epidemic_episodes.csv', index_col=0)
df.DATA = pd.to_datetime(df.DATA)
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] = df['CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] / df['POPULACAO_2022']  * 100000

In [3]:
df['MUNICIPIO_UF'] = df['MUNICIPIO'] + '/' + df['SIGLA_ESTADO']

In [4]:
df_final = df[df.DATA == df.DATA.max()]

In [5]:
df_results_municipalites = pd.read_csv('../data/execution_results.csv')

In [6]:
days_to_recovery = 8

## Extracting parameters

In [7]:
df_r0_mean = pd.DataFrame()
df_ifr_mean = pd.DataFrame()
df_Omega_mean = pd.DataFrame()

df_r0_general = pd.DataFrame()
df_ifr_general = pd.DataFrame()
df_Omega_general = pd.DataFrame()

for municipality in df_results_municipalites.municipality.unique():    
    df_results = df_results_municipalites[df_results_municipalites.municipality == municipality]
    municipality_id = df_results.municipality_id.iloc[0]
    date_first_case = pd.to_datetime(df_results.date_first_case.unique()[0])
    
    df_covid_municipality = df[(df.MUNICIPIO == municipality) & (df.DATA >= date_first_case)]
    real_reproduction_number = df_covid_municipality.NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA.values
    real_new_deaths = df_covid_municipality.TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS.values
    real_total_deaths = df_covid_municipality.TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS.cumsum().values
    real_new_cases = df_covid_municipality.CASOS_NOVOS_MEDIA_MOVEL_7_DIAS.values
    real_total_cases = df_covid_municipality.CASOS_NOVOS_MEDIA_MOVEL_7_DIAS.cumsum().values
        
    df_S = pd.DataFrame()
    df_I = pd.DataFrame()
    df_R = pd.DataFrame()
    df_D = pd.DataFrame()
    df_I_accumulated = pd.DataFrame()
    df_new_deaths = pd.DataFrame()
    df_rt = pd.DataFrame()
    df_new_cases = pd.DataFrame()
    df_r0 = pd.DataFrame()
    df_IFR = pd.DataFrame()
    df_Omega = pd.DataFrame()
    list_fuzzy_fast_transition = []
    list_fuzzy_slow_transition = []
    
    for index, result in df_results.iterrows():
        result = result.dropna()
        
        y = get_sirds(result)
        S, I, R, D, I_accumulated = y
        min_length = min(len(real_new_deaths), len(D))
        S = S[:min_length]
        I = I[:min_length]
        R = R[:min_length]
        D = D[:min_length]
        I_accumulated = I_accumulated[:min_length]
        N = S[0] + I[0] + R[0] + D[0]
        
        dates = df_covid_municipality[:min_length].DATA.values
        
        D_new_deaths, reproduction_number_sird, I_new_cases, epidemic_periods_with_fast_transition_fuzzy_variable, epidemic_periods_with_slow_transition_fuzzy_variable, days_between_infections_values, case_fatality_probability_values, loss_immunity_in_days_values, estimated_days_between_infections, estimated_case_fatality_probability, estimated_loss_immunity_in_days = get_sirds_extras(result, S, D, I_accumulated)
        
        df_S_temp = pd.DataFrame({'date': dates, 'S': S})
        df_I_temp = pd.DataFrame({'date': dates, 'I': I})
        df_R_temp = pd.DataFrame({'date': dates, 'R': R})
        df_D_temp = pd.DataFrame({'date': dates, 'D': D})
        df_I_accumulated_temp = pd.DataFrame({'date': dates, 'I_accumulated': I_accumulated})
        df_new_deaths_temp = pd.DataFrame({'date': dates[1:], 'deaths': D_new_deaths})
        df_rt_temp = pd.DataFrame({'date': dates, 'rt': reproduction_number_sird})
        df_new_cases_temp = pd.DataFrame({'date': dates[1:], 'cases': I_new_cases})
        df_r0_temp = pd.DataFrame({'date': dates, 'r0': days_to_recovery/estimated_days_between_infections})
        df_r0_temp['index'] = index
        df_IRF_temp = pd.DataFrame({'date': dates, 'ifr': estimated_case_fatality_probability})
        df_IRF_temp['index'] = index
        df_Omega_temp = pd.DataFrame({'date': dates, 'Omega': estimated_loss_immunity_in_days})
        df_Omega_temp['index'] = index
                   
        df_S = pd.concat([df_S, df_S_temp])
        df_I = pd.concat([df_I, df_I_temp])
        df_R = pd.concat([df_R, df_R_temp])
        df_D = pd.concat([df_D, df_D_temp])
        df_I_accumulated = pd.concat([df_I_accumulated, df_I_accumulated_temp])
        df_new_deaths = pd.concat([df_new_deaths, df_new_deaths_temp])
        df_rt = pd.concat([df_rt, df_rt_temp])
        df_new_cases = pd.concat([df_new_cases, df_new_cases_temp])
        df_r0 = pd.concat([df_r0, df_r0_temp])
        df_IFR = pd.concat([df_IFR, df_IRF_temp])
        df_Omega = pd.concat([df_Omega, df_Omega_temp])
        list_fuzzy_fast_transition.append(epidemic_periods_with_fast_transition_fuzzy_variable)
        list_fuzzy_slow_transition.append(epidemic_periods_with_slow_transition_fuzzy_variable)

    df_r0['municipality_id'] = municipality_id
    df_r0['municipality'] = municipality
    df_IFR['municipality_id'] = municipality_id
    df_IFR['municipality'] = municipality
    df_Omega['municipality_id'] = municipality_id
    df_Omega['municipality'] = municipality

    df_r0_mean_municipality = df_r0.groupby(['municipality_id', 'municipality', 'date']).mean().reset_index()
    df_r0_mean = pd.concat([df_r0_mean, df_r0_mean_municipality], ignore_index=True)
    
    df_ifr_mean_municipality = df_IFR.groupby(['municipality_id', 'municipality', 'date']).mean().reset_index()
    df_ifr_mean = pd.concat([df_ifr_mean, df_ifr_mean_municipality], ignore_index=True)    
    
    df_Omega_mean_municipality = df_Omega.groupby(['municipality_id', 'municipality', 'date']).mean().reset_index()
    df_Omega_mean = pd.concat([df_Omega_mean, df_Omega_mean_municipality], ignore_index=True)
    
    df_ifr_general = pd.concat([df_ifr_general, df_IFR], ignore_index=True)
    df_r0_general = pd.concat([df_r0_general, df_r0], ignore_index=True)
    df_Omega_general = pd.concat([df_Omega_general, df_Omega], ignore_index=True)

## Saving data

In [8]:
df_r0_mean.to_csv('../data/df_r0_mean.csv', index=False)
df_ifr_mean.to_csv('../data/df_ifr_mean.csv', index=False)
df_Omega_mean.to_csv('../data/df_Omega_mean.csv', index=False)

df_r0_general.to_csv('../data/df_r0_general.csv', index=False)
df_ifr_general.to_csv('../data/df_ifr_general.csv', index=False)
df_Omega_general.to_csv('../data/df_Omega_general.csv', index=False)